In [1]:
from dotenv import load_dotenv
import os
from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
load_dotenv()


api_key = os.getenv("AZURE_API_KEY")
azure_endpoint = os.getenv("AZURE_ENDPOINT")

Settings.llm = AzureOpenAI(
    model="gpt-4-32k",
    deployment_name="gpt4-demetrius",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version="2023-07-01-preview",
)

In [2]:
import subprocess
import os

def run_command_in_directory(cwd:str, command:str):
    """Use this function to run shell commands in a directory

    Args:
        cwd (str): The working directory where the command needs to be run
        command (str): the command that has to be run

    Returns:
        stdout (str): the shell output if the command is successfully run
        stderr (str): the shell output error if the command is not successfully run 
    """
    try:
        os.chdir(cwd)
        
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        
        return result.stdout, result.stderr
    except Exception as e:
        return None, str(e)
    
def write_to_file(file_path: str, content: str):
    """Use this function to write content to a file

    Args:
        file_path (str): The path to the file where the content needs to be written
        content (str): The content that has to be written to the file

    Returns:
        success (bool): True if the content is successfully written to the file
        error (str): The error message if there is an issue with writing to the file
    """
    try:
        with open(file_path, 'w') as file:
            file.write(content)
        return True, None
    except Exception as e:
        return False, str(e)




In [3]:
from llama_index.core.tools import FunctionTool
shellTool = FunctionTool.from_defaults(fn=run_command_in_directory)
writeTool = FunctionTool.from_defaults(fn=write_to_file)

In [4]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [shellTool,writeTool], 
    verbose=True
)
agent = AgentRunner(agent_worker,verbose=True)

In [5]:
#reqs for the project
requirements = """I want a backend that satisfies the following requirements
It should have an endpoint(getStudentDetails) that gets all the student details(name,rollno,year,dob) from the mongoURL: mongodb://localhost:27017/
It should have an endpoint to delete a student in the mongo db with his rollno
It should have an endpoint to get students from a particular year
"""
cwd = r"F:\PROJECTS\web-automation\test2"

In [6]:
#selecting which framework to use for the project
framework = agent.query(f"Suggest a framework alone for the following requirements\nUser:\n{requirements}")
framework

> Running step 5de96eff-5e99-40ff-8581-26696a178369. Step input: Suggest a framework alone for the following requirements
User:
I want a backend that satisfies the following requirements
It should have an endpoint(getStudentDetails) that gets all the student details(name,rollno,year,dob) from the mongoURL: mongodb://localhost:27017/
It should have an endpoint to delete a student in the mongo db with his rollno
It should have an endpoint to get students from a particular year

Added user message to memory: Suggest a framework alone for the following requirements
User:
I want a backend that satisfies the following requirements
It should have an endpoint(getStudentDetails) that gets all the student details(name,rollno,year,dob) from the mongoURL: mongodb://localhost:27017/
It should have an endpoint to delete a student in the mongo db with his rollno
It should have an endpoint to get students from a particular year

=== LLM Response ===
Based on your requirements, I would suggest using th

Response(response="Based on your requirements, I would suggest using the Node.js framework with Express.js for creating your backend API endpoints. \n\nExpress.js is a minimal and flexible Node.js web application framework that provides a robust set of features for web and mobile applications. It is very easy to create RESTful APIs using Express.js.\n\nFor interacting with MongoDB, you can use Mongoose. Mongoose is an Object Data Modeling (ODM) library for MongoDB and Node.js. It manages relationships between data, provides schema validation, and is used to translate between objects in code and the representation of those objects in MongoDB.\n\nHere is a basic structure of how your endpoints could look like:\n\n```javascript\nconst express = require('express');\nconst mongoose = require('mongoose');\nconst app = express();\n\nmongoose.connect('mongodb://localhost:27017/', {useNewUrlParser: true, useUnifiedTopology: true});\n\nconst studentSchema = new mongoose.Schema({\n  name: String,

In [7]:
#initializing the folder
init = agent.query(f"Requirements:\n{requirements}\n\ninitialize the folder ({cwd}) and install the dependencies for the project for the framework({framework}). Return a summary of what u done")
init

> Running step 12b2825b-b35f-4622-b7f1-a322c6dcfb7f. Step input: Requirements:
I want a backend that satisfies the following requirements
It should have an endpoint(getStudentDetails) that gets all the student details(name,rollno,year,dob) from the mongoURL: mongodb://localhost:27017/
It should have an endpoint to delete a student in the mongo db with his rollno
It should have an endpoint to get students from a particular year


initialize the folder (F:\PROJECTS\web-automation\test2) and install the dependencies for the project for the framework(Based on your requirements, I would suggest using the Node.js framework with Express.js for creating your backend API endpoints. 

Express.js is a minimal and flexible Node.js web application framework that provides a robust set of features for web and mobile applications. It is very easy to create RESTful APIs using Express.js.

For interacting with MongoDB, you can use Mongoose. Mongoose is an Object Data Modeling (ODM) library for MongoDB a

APIConnectionError: Connection error.

In [ ]:
#writing the content in the files that are needed
result = agent.query(f"Requirements:\n{requirements}\n\nThe folder is initialized for {framework} and the job yet done is {init}. I want you to write all the files in {cwd} to acheive the requirements mentioned above")
result